*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Model Ensembling

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This notebook provides an example of how to ensemble multiple recommendation models in Cornac.

Ensemble models is a technique that combines the predictions of multiple models to produce a single prediction. The idea is that by combining the predictions of multiple models, we can improve the overall performance of the recommendation system.

We will use the MovieLens 100K dataset and ensemble 2 models.

** Note: ** This notebook requires the `scikit-learn` package. 

## 1. Setup

### Install required dependencies

In [1]:
! pip install seaborn scikit-learn

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from cornac.datasets import movielens
from cornac.data import Dataset
from cornac.models import BPR, WMF
from cornac.eval_methods import RatioSplit
from cornac.metrics import Precision, Recall
from cornac.utils import cache
from cornac import Experiment

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

/Users/darrylong/miniconda3/envs/gcmc-pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Prepare Experiment

### Loading Dataset

First, we load the MovieLens 100K dataset.

In [2]:
data = movielens.load_feedback(variant="100K") # load dataset
# dataset = Dataset.from_uir(data) # convert to Dataset object

rs = RatioSplit(data, test_size=0.2, seed=42, verbose=True)
train_set, test_set = rs.train_set, rs.test_set

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


### Training BPR and WMF models

We will train two models: 

1. BPR (Bayesian Personalized Ranking)
2. WMF (Weighted Matrix Factorization)

In [3]:
# Train BPR model
bpr_model = BPR(k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001)
# Train WMF model
wmf_model = WMF(k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,)

models = [bpr_model, wmf_model]
metrics = [Precision(k=10), Recall(k=10)]

experiment = Experiment(rs, models, metrics, user_based=True).run()


[BPR] Training started!

[BPR] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1428.12it/s]



[WMF] Training started!


100%|██████████| 300/300 [00:24<00:00, 12.15it/s, loss=173]


Learning completed!

[WMF] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:01<00:00, 699.13it/s]


TEST:
...
    | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------------ + --------- + --------- + --------
BPR |       0.3139 |    0.1995 |    1.7123 |   0.6646
WMF |       0.3079 |    0.2016 |  114.3336 |   1.3625



In [5]:
# X_train, y_train = list(zip(train_set.uir_tuple[0], train_set.uir_tuple[1])), train_set.uir_tuple[2]
# X_test, y_test = list(zip(test_set.uir_tuple[0], test_set.uir_tuple[1])), test_set.uir_tuple[2]

# bpr_scores = [bpr_model.score(uidx)[iidx] for uidx, iidx in list(X_test)]
# wmf_scores = [wmf_model.score(uidx)[iidx] for uidx, iidx in X_test]

# df = pd.DataFrame({'user': test_set.uir_tuple[0], 'item': test_set.uir_tuple[1], 'groundtruth rating': test_set.uir_tuple[2], 'bpr_rating': bpr_scores, 'wmf_rating': wmf_scores})
# df.head()


### Interpreting Results

In [6]:
# Download some information of MovieLens 100K dataset
item_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"), 
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL", 
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", 
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", 
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])
     

In [93]:
from IPython.display import display

item_idx2id = list(train_set.item_ids)

# take a step back and take a look at training data
# make sense of the data and recommendations

training_data_df = pd.DataFrame(zip(*train_set.uir_tuple))
training_data_df.columns = ['user_idx', 'item_idx', 'rating']
training_data_df['item_id'] = training_data_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1)

UIDX = 3
TOPK = 50

filter_df = training_data_df[(training_data_df['rating'] == 5.0) & (training_data_df['user_idx'] == UIDX)]
filter_df = item_df.loc[[int(item_id) for item_id in filter_df["item_id"]]]
filter_df = filter_df.select_dtypes(np.number).sum() # sum number of movies in training set by movie genre
filter_df = filter_df.to_frame("Sum")
filter_df["%"] = filter_df["Sum"] / filter_df["Sum"].sum() * 100
filter_df["%"] = filter_df["%"].round(1)

# sum of genres in training data
print("Movies rated 5.0 by user index 3 in training data")
display(filter_df.sort_values("Sum", ascending=False))

Movies rated 5.0 by user index 3 in training data


,Sum,%
Drama,71,24.9
Comedy,39,13.7
Romance,32,11.2
Action,30,10.5
Thriller,29,10.2
Adventure,19,6.7
War,15,5.3
Crime,12,4.2
Sci-Fi,9,3.2
Mystery,8,2.8


As shown above in the training data, the top genres for user index 3 with movies rated 5.0 include 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller'.

Let's take a look at what movies and its coresponding genres are recommended to this user by the BPR and WMF models respectively.

In [123]:
# Picking out only the top 5 genres
top_genres = filter_df.sort_values("Sum", ascending=False).head(5).index.tolist()
print("\nTop 5 Genres in training data:", top_genres)

# Get top K recommendations for BPR and WMF
bpr_recommendations, bpr_scores = bpr_model.rank(UIDX)
wmf_recommendations, wmf_scores = wmf_model.rank(UIDX)

# Top K recommended items for each model
bpr_topk = [item_idx2id[iidx] for iidx in bpr_recommendations[:TOPK]]
wmf_topk = [item_idx2id[iidx] for iidx in wmf_recommendations[:TOPK]]

# print("BPR Top K Item IDs:", bpr_topk)
# print("WMF Top K Item IDs:", wmf_topk)

bpr_df = item_df.loc[[int(iid) for iid in bpr_topk]]
wmf_df = item_df.loc[[int(iid) for iid in wmf_topk]]

display("BPR: Top recommendations", bpr_df[["Title"] + top_genres].head(10))
display("WMF: Top recommendations", wmf_df[["Title"] + top_genres].head(10))

combined_df = pd.DataFrame({
    "Train Data %": filter_df["%"],
    "BPR Sum": bpr_df.select_dtypes(np.number).sum(),
    "WMF Sum": wmf_df.select_dtypes(np.number).sum()
})

combined_df['BPR %'] = combined_df['BPR Sum'] / combined_df['BPR Sum'].sum() * 100

combined_df["WMF %"] = combined_df["WMF Sum"] / combined_df["WMF Sum"].sum() * 100

combined_df = combined_df.round(1)
combined_df = combined_df.sort_values("Train Data %", ascending=False)

display("Train Data to Recommended % Distribution", combined_df[['Train Data %', 'BPR %', 'WMF %']])

# Show count of genres and percentage of genres in top K recommendations
# print("Movie Genre distribution in BPR")
# display(combined_df[["BPR Sum", "BPR %"]].sort_values(by="BPR Sum", ascending=False))

# print("Movie Genre distribution in WMF")
# display(combined_df[["WMF Sum", "WMF %"]].sort_values(by="WMF Sum", ascending=False))


# df_dict = dict(
#     list(
#         bpr_df.groupby(bpr_df.index)
#     )
# )

# display(bpr_df)

# display((bpr_df == 1).any())

# display(bpr_df.columns[(bpr_df == 1)])
# item_df['test'] = item_df.columns[(item_df.loc[[int(iid) for iid in bpr_topk]]).values == 1].tolist()


# print("\nTop K BPR Item recommendations with Movie Genre category names")
# display(item_df.loc[[int(iid) for iid in bpr_topk]][["Title"] + top_genres])

# print("\nTop K WMF Item recommendations with Movie Genre")
# display(item_df.loc[[int(iid) for iid in wmf_topk]][["Title"] + top_genres])


Top 5 Genres in training data: ['Drama', 'Comedy', 'Romance', 'Action', 'Thriller']


'BPR: Top recommendations'

,Title,Drama,Comedy,Romance,Action,Thriller
ItemID,,,,,,
215,Field of Dreams (1989),1,0,0,0,0
245,"Devil's Own, The (1997)",1,0,0,1,1
402,Ghost (1990),0,1,1,0,1
69,Forrest Gump (1994),0,1,1,0,0
315,Apt Pupil (1998),1,0,0,0,1
255,My Best Friend's Wedding (1997),0,1,1,0,0
328,Conspiracy Theory (1997),0,0,1,1,1
216,When Harry Met Sally... (1989),0,1,1,0,0
210,Indiana Jones and the Last Crusade (1989),0,0,0,1,0


'WMF: Top recommendations'

,Title,Drama,Comedy,Romance,Action,Thriller
ItemID,,,,,,
318,Schindler's List (1993),1,0,0,0,0
313,Titanic (1997),1,0,1,1,0
64,"Shawshank Redemption, The (1994)",1,0,0,0,0
127,"Godfather, The (1972)",1,0,0,1,0
215,Field of Dreams (1989),1,0,0,0,0
216,When Harry Met Sally... (1989),0,1,1,0,0
191,Amadeus (1984),1,0,0,0,0
735,Philadelphia (1993),1,0,0,0,0
196,Dead Poets Society (1989),1,0,0,0,0


'Train Data to Recommended % Distribution'

,Train Data %,BPR %,WMF %
Drama,24.9,18.2,32.4
Comedy,13.7,19.1,9.3
Romance,11.2,18.2,13.0
Action,10.5,10.9,10.2
Thriller,10.2,8.2,11.1
Adventure,6.7,7.3,4.6
War,5.3,4.5,6.5
Crime,4.2,0.9,2.8
Sci-Fi,3.2,4.5,4.6
Mystery,2.8,2.7,0.9


It seems that the BPR model tend to recommend the 'Comedy' and 'Romance' genres, while the WMF model tend to recommend mainly the 'Drama' genre.

Let's see what happens when we ensemble these two models. 

## 2. Simple model ensembling by Borda Count

We will ensemble the two models using the Borda Count method. The Borda Count method is a simple voting method that ranks the items based on the sum of their ranks from each model.

Assuming that we have a list of 5 items, the Borda Count method works as follows:

1. For each model, rank the items from 1 to 5 based on the predicted scores.
2. Sum the ranks of each item across all models.
3. Sort the items based on the sum of their ranks.
4. The top-ranked item is the final recommendation.
5. Repeat the process for the next user.

Given the below example for a random user 123:

| Rank | Model 1 | Model 2 | Model 3 | Allocated Points (N - rank) |
|------|---------|---------|---------|-----------------------------|
| 1    | A       | D       | E       | 5 - 1 = 4                   |
| 2    | B       | C       | A       | 5 - 2 = 3                   |
| 3    | C       | A       | B       | 5 - 3 = 2                   |
| 4    | D       | B       | D       | 5 - 4 = 1                   |
| 5    | E       | E       | C       | 5 - 5 = 0                   |

The final ranking is as follows after summing them all up

| Item | Total Points     |
|------|------------------|
| A    | 4 + 2 + 3 = 9    |
| B    | 3 + 1 + 2 = 6    |
| D    | 1 + 4 + 1 = 6    |
| C    | 2 + 3 + 0 = 5    |
| E    | 0 + 0 + 4 = 4    |

New ranking: A > B, D > C > E


Lets implement this method below by using pandas `DataFrame` for data manipulation.

In [118]:
# todo: group by users

rank_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

total_items = len(rank_df) # 1651 items

# Obtain ranks of the items based on the scores
rank_df["BPR Score"] = bpr_scores
rank_df["BPR Rank"] = rank_df["BPR Score"].rank(ascending=False).astype(int)
rank_df["BPR Inverse Rank"] = total_items - rank_df["BPR Rank"]

rank_df["WMF Score"] = wmf_scores
rank_df["WMF Rank"] = rank_df["WMF Score"].rank(ascending=False).astype(int)
rank_df["WMF Inverse Rank"] = total_items - rank_df["WMF Rank"]

# Get Borda Score by summing up points. Points are calculated as (total items - rank)
rank_df["Borda Count"] = rank_df["BPR Inverse Rank"] + rank_df["WMF Inverse Rank"] # Borda count

# Round decimal places for readability purposes
rank_df = rank_df.round(3)

display(rank_df)

,ItemID,BPR Score,BPR Rank,BPR Inverse Rank,WMF Score,WMF Rank,WMF Inverse Rank,Borda Count
0,381,1.365,452,1199,3.691,228,1423,2622
1,602,0.315,701,950,1.829,798,853,1803
2,431,1.168,491,1160,2.708,489,1162,2322
3,875,1.471,432,1219,2.368,601,1050,2269
4,182,1.949,338,1313,3.997,156,1495,2808
...,...,...,...,...,...,...,...,...
1646,1635,-2.562,1434,217,0.459,1354,297,514
1647,1650,-2.679,1489,162,0.613,1290,361,523
1648,1647,-2.612,1456,195,0.455,1362,289,484
1649,1663,-2.642,1468,183,0.239,1481,170,353


Now that we have a joint score, let's rerank this list and to provide the ensembled model's recommendation.

In [127]:
reranked_df = rank_df.sort_values("Borda Count", ascending=False)

display("Re-ranked Top K Item recommendations", reranked_df)

borda_count_topk = reranked_df["ItemID"].values[:TOPK]
# print("Top K Ensembled Item recommendations", borda_count_topk)

borda_df = item_df.loc[[int(i) for i in borda_count_topk]]

combined_df["Borda Count Sum"] = borda_df.select_dtypes(np.number).sum()
combined_df["Borda Count %"] = combined_df["Borda Count Sum"] / combined_df["Borda Count Sum"].sum() * 100
combined_df["Borda Count %"] = combined_df["Borda Count %"].round(1)

display("Borda Count Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %"]])

'Re-ranked Top K Item recommendations'

,ItemID,BPR Score,BPR Rank,BPR Inverse Rank,WMF Score,WMF Rank,WMF Inverse Rank,Borda Count
604,215,5.974,1,1650,5.433,5,1646,3296
308,216,4.911,8,1643,5.383,6,1645,3288
126,69,5.124,4,1647,5.236,12,1639,3286
387,315,5.022,5,1646,5.069,14,1637,3283
152,313,4.481,17,1634,6.064,2,1649,3283
...,...,...,...,...,...,...,...,...
1351,1536,-3.214,1623,28,0.031,1618,33,61
1443,1156,-3.235,1629,22,-0.003,1628,23,45
1605,1347,-3.173,1615,36,-0.157,1651,0,36
1507,1332,-3.478,1646,5,0.006,1626,25,30


'Borda Count Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %
Drama,24.9,18.2,32.4,23.0
Comedy,13.7,19.1,9.3,13.3
Romance,11.2,18.2,13.0,14.2
Action,10.5,10.9,10.2,9.7
Thriller,10.2,8.2,11.1,9.7
Adventure,6.7,7.3,4.6,8.0
War,5.3,4.5,6.5,6.2
Crime,4.2,0.9,2.8,1.8
Sci-Fi,3.2,4.5,4.6,3.5
Mystery,2.8,2.7,0.9,2.7


Now, it seems that the ensembled model is able to provide a more diverse set of recommendations. Recommendations from movie genres such as 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller' are now included in the top 10 recommendations. >> Well balanced recommendations

In the next section, we will see how we could further add more models to the ensemble.

## 3. Adding more models to the Borda Count ensemble

We can easily add more models to the ensemble by training them and adding them. One approach is to train a model with different initializations using different random seeds. By adding multiple similar models of different random seeds (`seed=123`), some models could perform better for a set of users, while other models could perform better for another set of users.

By ensembling these models, we could potentially achieve better performance when combined.

Let's try adding a few more similar models with different random seed initializations.

In [129]:
# BPR models with different seeds
# bpr_seed_123 = BPR(name="BPR_123", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=123)
# bpr_seed_456 = BPR(name="BPR_456", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=456)
# bpr_seed_789 = BPR(name="BPR_789", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=789)
# bpr_seed_888 = BPR(name="BPR_888", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=888)
# bpr_seed_999 = BPR(name="BPR_999", k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=999)

# WMF models with different seeds
wmf_model_123 = WMF(name="WMF_123", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=123)
wmf_model_456 = WMF(name="WMF_456", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=456)
wmf_model_789 = WMF(name="WMF_789", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=789)
wmf_model_888 = WMF(name="WMF_888", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=888)
wmf_model_999 = WMF(name="WMF_999", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=999)

models = [wmf_model_123, wmf_model_456, wmf_model_789, wmf_model_888, wmf_model_999]

experiment = Experiment(rs, models, metrics, user_based=True).run()


[WMF_123] Training started!


100%|██████████| 300/300 [00:16<00:00, 17.88it/s, loss=173]


Learning completed!

[WMF_123] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1924.79it/s]



[WMF_456] Training started!


...


[WMF_999] Training started!


100%|██████████| 300/300 [00:17<00:00, 16.71it/s, loss=171]


Learning completed!

[WMF_999] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1719.80it/s]


TEST:
...
        | Precision@10 | Recall@10 | Train (s) | Test (s)
------- + ------------ + --------- + --------- + --------
WMF_123 |       0.3131 |    0.2081 |   17.1978 |   0.4942
WMF_456 |       0.3033 |    0.1997 |   16.5709 |   0.4557
WMF_789 |       0.3094 |    0.2014 |   16.4109 |   0.4861
WMF_888 |       0.3106 |    0.2041 |   16.2947 |   0.5635
WMF_999 |       0.3119 |    0.2035 |   18.1191 |   0.5538



Based on the results, we can see that even within the same model, the results can vary. 

Let's try ensembling all these models together into 1 single model by Borda Count, and look at its recommendations.

In [137]:
rank_2_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

rank_2_df["Enhanced Borda Count"] = 0

for model in models:
    name = model.name
    recommendations, scores = model.rank(UIDX)
    rank_2_df[name + "_rating"] = scores
    rank_2_df[name + "_rank"] = rank_2_df[name + "_rating"].rank(ascending=False).astype(int)
    rank_2_df[name + "_inverse_rank"] = total_items - rank_2_df[name + "_rank"]
    rank_2_df["Enhanced Borda Count"] = rank_2_df["Enhanced Borda Count"] + rank_2_df[name + "_inverse_rank"]

rank_2_df = rank_2_df.round(3)

print("Model score calculation:")
display(rank_2_df[["WMF_123_inverse_rank", "WMF_456_inverse_rank", "WMF_789_inverse_rank", "WMF_888_inverse_rank", "WMF_999_inverse_rank", "Enhanced Borda Count"]].sort_values("Enhanced Borda Count", ascending=False).head(10))

# print("\nRe-ranked Top K Item recommendations")
enhanced_borda_count_topk = list(rank_2_df.sort_values("Enhanced Borda Count", ascending=False)["ItemID"].values[:TOPK])
enhanced_borda_df = item_df.loc[[int(i) for i in enhanced_borda_count_topk]]

combined_df["Enhanced Borda Count Sum"] = enhanced_borda_df.select_dtypes(np.number).sum()
combined_df["Enhanced Borda Count %"] = combined_df["Enhanced Borda Count Sum"] / combined_df["Enhanced Borda Count Sum"].sum() * 100
combined_df["Enhanced Borda Count %"] = combined_df["Enhanced Borda Count %"].round(1)

display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Enhanced Borda Count %"]])

Model score calculation:


,WMF_123_inverse_rank,WMF_456_inverse_rank,WMF_789_inverse_rank,WMF_888_inverse_rank,WMF_999_inverse_rank,Enhanced Borda Count
152,1650,1650,1650,1650,1650,8250
132,1637,1649,1649,1648,1647,8230
92,1612,1646,1645,1649,1649,8201
156,1644,1620,1648,1637,1638,8187
197,1631,1644,1641,1635,1634,8185
37,1646,1599,1647,1643,1646,8181
305,1582,1645,1640,1647,1648,8162
279,1613,1637,1646,1632,1631,8159
522,1638,1613,1632,1640,1635,8158
239,1641,1632,1622,1641,1608,8144


'Enhanced Borda Count Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Enhanced Borda Count %
Drama,24.9,18.2,32.4,23.0,21.4
Comedy,13.7,19.1,9.3,13.3,16.2
Romance,11.2,18.2,13.0,14.2,19.7
Action,10.5,10.9,10.2,9.7,8.5
Thriller,10.2,8.2,11.1,9.7,6.0
Adventure,6.7,7.3,4.6,8.0,4.3
War,5.3,4.5,6.5,6.2,6.8
Crime,4.2,0.9,2.8,1.8,0.9
Sci-Fi,3.2,4.5,4.6,3.5,5.1
Mystery,2.8,2.7,0.9,2.7,1.7


## 4. Advanced Model Ensembling

We could continue by thinking of this as a meta-learning problem. We could treat recommendations of each base model as features and train a meta-learner to predict the final recommendation.

This could be any ML model such as a Linear Regression, Random Forest, Gradient Boosting, or even a Neural Network.


In this example, we will use a simple Linear Regression model to predict the final recommendation.

In [142]:
# We continue using UIDX = 3 and TOPK = 50
rank_3_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

# print(len(item_idx2id))
# print(train_set.uir_tuple[0])
# print(train_set.uir_tuple[1])

# Lets get all the scores for the models trained in Part 3.
training_df = pd.DataFrame(zip(*train_set.uir_tuple))
training_df.columns = ['user_idx', 'item_idx', 'ground_rating']
test_df = pd.DataFrame(zip(*test_set.uir_tuple))
test_df.columns = ['user_idx', 'item_idx', 'ground_rating']
test_df['item_id'] = test_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1)

for model in models:
    name = model.name
    training_df[name + "_rating"] = training_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1)
    test_df[name + "_rating"] = test_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1)

# display(training_df.tail(10))

X_train = training_df[['WMF_123_rating', 'WMF_456_rating', 'WMF_789_rating', 'WMF_888_rating', 'WMF_999_rating']]
y_train = training_df['ground_rating']
X_test = test_df[['WMF_123_rating', 'WMF_456_rating', 'WMF_789_rating', 'WMF_888_rating', 'WMF_999_rating']]

from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

test_df["Linear Regression Rating"] = y_pred

# filter by predicted rating
UIDX = 3
TOPK = 50

test_df = test_df.sort_values("Linear Regression Rating", ascending=False)

top_item_ids = test_df[test_df['user_idx'] == UIDX]['item_id'].values[:TOPK]
# print("Top K Item recommendations")
# print(top_item_ids)

# print("\nTop K Item recommendations with Movie Genre")
linear_regression_df = item_df.loc[[int(i) for i in top_item_ids]]
combined_df["Linear Regression Sum"] = linear_regression_df.select_dtypes(np.number).sum()
combined_df["Linear Regression %"] = combined_df["Linear Regression Sum"] / combined_df["Linear Regression Sum"].sum() * 100
combined_df["Linear Regression %"] = combined_df["Linear Regression %"].round(1)

display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Enhanced Borda Count %", "Linear Regression %"]])




'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Enhanced Borda Count %,Linear Regression %
Drama,24.9,18.2,32.4,23.0,21.4,19.3
Comedy,13.7,19.1,9.3,13.3,16.2,12.8
Romance,11.2,18.2,13.0,14.2,19.7,15.6
Action,10.5,10.9,10.2,9.7,8.5,11.9
Thriller,10.2,8.2,11.1,9.7,6.0,9.2
Adventure,6.7,7.3,4.6,8.0,4.3,5.5
War,5.3,4.5,6.5,6.2,6.8,2.8
Crime,4.2,0.9,2.8,1.8,0.9,2.8
Sci-Fi,3.2,4.5,4.6,3.5,5.1,1.8
Mystery,2.8,2.7,0.9,2.7,1.7,2.8


To reiterate, the top genres for user index 3 with movies rated 5.0 include 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller'.

### Applying Random Forest model



In [143]:
randomforest_model = RandomForestRegressor(n_estimators=200, random_state=42) # sklearn Random Forest model
randomforest_model.fit(X_train, y_train)

y_pred = randomforest_model.predict(X_test)

UIDX = 3
TOPK = 50

test_df["Random Forest Rating"] = y_pred

# Sort the items based on the score
test_df = test_df.sort_values("Random Forest Rating", ascending=False)

top_item_ids = test_df[test_df['user_idx'] == UIDX]['item_id'].values[:TOPK]
# print("Top K Item recommendations")
# print(top_item_ids)

# print("\nTop K Item recommendations with Movie Genre")
# display(item_df.loc[[int(i) for i in top_item_ids]])
random_forest_df = item_df.loc[[int(i) for i in top_item_ids]]
combined_df["Random Forest Sum"] = random_forest_df.select_dtypes(np.number).sum()
combined_df["Random Forest %"] = combined_df["Random Forest Sum"] / combined_df["Random Forest Sum"].sum() * 100
combined_df["Random Forest %"] = combined_df["Random Forest %"].round(1)

display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Enhanced Borda Count %", "Linear Regression %", "Random Forest %"]])



'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Enhanced Borda Count %,Linear Regression %,Random Forest %
Drama,24.9,18.2,32.4,23.0,21.4,19.3,29.5
Comedy,13.7,19.1,9.3,13.3,16.2,12.8,11.6
Romance,11.2,18.2,13.0,14.2,19.7,15.6,10.5
Action,10.5,10.9,10.2,9.7,8.5,11.9,15.8
Thriller,10.2,8.2,11.1,9.7,6.0,9.2,8.4
Adventure,6.7,7.3,4.6,8.0,4.3,5.5,6.3
War,5.3,4.5,6.5,6.2,6.8,2.8,2.1
Crime,4.2,0.9,2.8,1.8,0.9,2.8,3.2
Sci-Fi,3.2,4.5,4.6,3.5,5.1,1.8,1.1
Mystery,2.8,2.7,0.9,2.7,1.7,2.8,2.1
